# Requisitos
Certifique-se de que os pacotes necessários estão instalados. Execute o seguinte comando:
```bash
pip install pandas pyarrow parquet

In [1]:
# importar os pacotes necessários
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import os
import time

In [11]:
# função para formatar o tamanho do arquivo
def convert_bytes(num):
    """
    this function will convert bytes to MB.... GB... etc
    """
    for x in ['bytes', 'KB', 'MB', 'GB', 'TB']:
        if num < 1024.0:
            return "%3.1f %s" % (num, x)
        num /= 1024.0

# fonte : https://stackoverflow.com/questions/2104080/how-do-i-check-file-size-in-python

In [12]:
# Caminho do arquivo CSV original
csv_file_path = '../../databases/csv/PFW_2021_public.csv'

# Caminho de destino onde os arquivos Parquet serão armazenados
output_dir = 'dataset/compression'
os.makedirs(output_dir, exist_ok=True)

# Caminhos para os arquivos Parquet com diferentes compressões
input_parquet = os.path.join(output_dir, 'sem_compressao.parquet')

# Caminho para o arquivo Parquet com compressão Snappy
output_parquet_snappy = os.path.join(output_dir, 'snappy.parquet')

## Compressão de arquivo CSV em Parquet

In [21]:
def criar_parquet_sem_compressao(csv_path, output_path, tamanho_csv=None):
    """
    Lê um CSV, converte para Parquet sem compressão, mede o tempo de execução,
    o tamanho do arquivo Parquet gerado e calcula a taxa de compressão em relação ao CSV.

    Args:
        csv_path (str): Caminho para o arquivo CSV.
        output_path (str): Caminho de saída para o arquivo Parquet.
        tamanho_csv (int, optional): Tamanho do arquivo CSV original para calcular a taxa de compressão.
    """
    try:
        # Lê o arquivo CSV para um DataFrame
        df = pd.read_csv(csv_path)

        # Converte o DataFrame para uma Tabela PyArrow
        table = pa.Table.from_pandas(df)

        # Medir o tempo de execução para escrever o arquivo Parquet
        start_time = time.time()
        pq.write_table(table, output_path, compression=None)  # Sem compressão
        end_time = time.time()

        # Obtém o tamanho do arquivo Parquet gerado
        tamanho_arquivo = os.path.getsize(output_path)

        # Exibe o tempo e o tamanho do arquivo
        print(f"Arquivo Parquet SEM compressão criado em {end_time - start_time:.4f} segundos.")
        print(f"Tamanho do arquivo: {convert_bytes(tamanho_arquivo)}")

        # Calcula a taxa de compressão se o tamanho do arquivo CSV for fornecido
        if tamanho_csv:
            taxa_compressao = (tamanho_csv - tamanho_arquivo) / tamanho_csv * 100
            print(f"Taxa de redução em relação ao CSV: {taxa_compressao:.2f}%")

    except FileNotFoundError:
        print(f"Erro: Arquivo '{csv_path}' não encontrado.")
    except pa.ArrowInvalid as e:
        print(f"Erro na conversão para PyArrow: {e}")
    except Exception as e:
        print(f"Ocorreu um erro ao escrever o Parquet: {e}")

def main():
    os.makedirs(output_dir, exist_ok=True)

    # Obtém o tamanho do arquivo CSV original para usar como base para a taxa de compressão
    try:
        tamanho_csv = os.path.getsize(csv_file_path)
        print(f"Tamanho do arquivo CSV original: {convert_bytes(tamanho_csv)}")
    except FileNotFoundError:
        print(f"Erro: Arquivo CSV original '{csv_file_path}' não encontrado. A taxa de compressão não será calculada.")
        tamanho_csv = None

    # Gerando o arquivo Parquet SEM compressão
    output_parquet_path_sem_compressao = os.path.join(output_dir, 'sem_compressao.parquet')
    criar_parquet_sem_compressao(csv_file_path, output_parquet_path_sem_compressao, tamanho_csv=tamanho_csv)

if __name__ == "__main__":
    main()


Tamanho do arquivo CSV original: 14.0 MB
Arquivo Parquet SEM compressão criado em 0.0571 segundos.
Tamanho do arquivo: 4.5 MB
Taxa de redução em relação ao CSV: 67.86%


## Compressão de arquivo Parquet com Snappy

In [23]:
def comprimir_parquet_snappy(input_path, output_path):
    """
    Comprime um arquivo Parquet usando Snappy.

    Args:
        input_path (str): Caminho para o arquivo Parquet de entrada.
        output_path (str): Caminho para o arquivo Parquet de saída (com compressão Snappy).
    """
    try:
        start_time = time.time()

        # Ler o arquivo Parquet usando PyArrow
        table = pq.read_table(input_path)

        # Escrever o arquivo Parquet com compressão Snappy
        pq.write_table(table, output_path, compression='snappy')

        end_time = time.time()

        tamanho_original = os.path.getsize(input_path)
        tamanho_comprimido = os.path.getsize(output_path)
        taxa_compressao = (tamanho_original - tamanho_comprimido) / tamanho_original * 100

        # Exibe o tempo e os tamanhos dos arquivos de forma legível
        print(f"Arquivo comprimido com Snappy em {end_time - start_time:.4f} segundos.")
        print(f"Tamanho original: {convert_bytes(tamanho_original)}")
        print(f"Tamanho comprimido: {convert_bytes(tamanho_comprimido)}")
        print(f"Taxa de compressão: {taxa_compressao:.2f}%")

    except FileNotFoundError:
        print(f"Erro: Arquivo '{input_path}' não encontrado.")
    except pa.ArrowInvalid as e:
        print(f"Erro ao ler o arquivo Parquet: {e}")
    except Exception as e:
        print(f"Ocorreu um erro durante a compressão: {e}")

def main():
    """Função principal."""
    output_dir = "output"
    os.makedirs(output_dir, exist_ok=True)

    if not os.path.exists(input_parquet):
        print(f"Erro: Arquivo Parquet original '{input_parquet}' não encontrado.")
        return

    # Chama a função para compressão com Snappy
    comprimir_parquet_snappy(input_parquet, output_parquet_snappy)

if __name__ == "__main__":
    main()


Arquivo comprimido com Snappy em 0.1126 segundos.
Tamanho original: 4.5 MB
Tamanho comprimido: 3.2 MB
Taxa de compressão: 30.01%


## Comparação de compressão em arquivos Parquet - snappy | gzip | brotli | zstd

In [25]:
def criar_parquet_com_compressao(csv_path, output_path, compression, tamanho_csv=None):
    """
    Lê um CSV, converte para Parquet com compressão especificada, mede o tempo de execução,
    o tamanho do arquivo Parquet gerado e calcula a taxa de compressão em relação ao CSV.

    Args:
        csv_path (str): Caminho para o arquivo CSV.
        output_path (str): Caminho de saída para o arquivo Parquet.
        compression (str): Tipo de compressão (snappy, gzip, brotli, zstd).
        tamanho_csv (int, optional): Tamanho do arquivo CSV original para calcular a taxa de compressão.
    """
    try:
        # Lê o arquivo CSV para um DataFrame
        df = pd.read_csv(csv_path)

        # Converte o DataFrame para uma Tabela PyArrow
        table = pa.Table.from_pandas(df)

        # Medir o tempo de execução para escrever o arquivo Parquet com compressão
        start_time = time.time()
        pq.write_table(table, output_path, compression=compression)
        end_time = time.time()

        # Obtém o tamanho do arquivo Parquet gerado
        tamanho_arquivo = os.path.getsize(output_path)

        # Exibe o tempo e o tamanho do arquivo
        print(f"Arquivo Parquet com compressão {compression} criado em {end_time - start_time:.4f} segundos.")
        print(f"Tamanho do arquivo: {convert_bytes(tamanho_arquivo)}")

        # Calcula a taxa de compressão se o tamanho do arquivo CSV for fornecido
        if tamanho_csv:
            taxa_compressao = (tamanho_csv - tamanho_arquivo) / tamanho_csv * 100
            print(f"Taxa de redução em relação ao CSV: {taxa_compressao:.2f}%")

    except FileNotFoundError:
        print(f"Erro: Arquivo '{csv_path}' não encontrado.")
    except pa.ArrowInvalid as e:
        print(f"Erro na conversão para PyArrow: {e}")
    except Exception as e:
        print(f"Ocorreu um erro ao escrever o Parquet: {e}")

def comparar_compression(csv_path, output_dir, tamanho_csv=None):
    """
    Compara os tipos de compressão: snappy, gzip, brotli, zstd.
    Para cada compressão, mede tempo, tamanho do arquivo e taxa de compressão.

    Args:
        csv_path (str): Caminho para o arquivo CSV.
        output_dir (str): Diretório de saída para os arquivos Parquet.
        tamanho_csv (int, optional): Tamanho do arquivo CSV original para cálculo da taxa de compressão.
    """
    # Tipos de compressão para comparar
    compressions = ['snappy', 'gzip', 'brotli', 'zstd']

    # Itera sobre cada tipo de compressão
    for compression in compressions:
        output_path = os.path.join(output_dir, f'{compression}.parquet')
        print(f"\nCriando Parquet com compressão {compression}...")

        # Chama a função de criação de Parquet para cada compressão
        criar_parquet_com_compressao(csv_path, output_path, compression, tamanho_csv=tamanho_csv)

def main():
    """Função principal."""
    os.makedirs(output_dir, exist_ok=True)

    # Obtém o tamanho do arquivo CSV original para usar como base para a taxa de compressão
    try:
        tamanho_csv = os.path.getsize(csv_file_path)
        print(f"Tamanho do arquivo CSV original: {convert_bytes(tamanho_csv)}")
    except FileNotFoundError:
        print(f"Erro: Arquivo CSV original '{csv_file_path}' não encontrado. A taxa de compressão não será calculada.")
        tamanho_csv = None

    # Comparar os arquivos Parquet com diferentes compressões
    comparar_compression(csv_file_path, output_dir, tamanho_csv=tamanho_csv)

if __name__ == "__main__":
    main()


Tamanho do arquivo CSV original: 14.0 MB

Criando Parquet com compressão snappy...
Arquivo Parquet com compressão snappy criado em 0.0718 segundos.
Tamanho do arquivo: 3.2 MB
Taxa de redução em relação ao CSV: 77.51%

Criando Parquet com compressão gzip...
Arquivo Parquet com compressão gzip criado em 0.2651 segundos.
Tamanho do arquivo: 2.4 MB
Taxa de redução em relação ao CSV: 82.87%

Criando Parquet com compressão brotli...
Arquivo Parquet com compressão brotli criado em 0.3697 segundos.
Tamanho do arquivo: 2.4 MB
Taxa de redução em relação ao CSV: 82.96%

Criando Parquet com compressão zstd...
Arquivo Parquet com compressão zstd criado em 0.0718 segundos.
Tamanho do arquivo: 2.4 MB
Taxa de redução em relação ao CSV: 82.99%


## Usando Compressão Automática com o PyArrow (sem especificar o tipo de compressão)

O PyArrow suporta compressão automática para Parquet se nenhum tipo de compressão for especificado. Por padrão, ele escolherá a melhor compressão disponível dependendo da instalação

In [27]:
def criar_parquet_com_compressao_automatica(csv_path, output_path):
    """
    Lê um CSV, converte para Parquet com compressão automática.

    Args:
        csv_path (str): Caminho para o arquivo CSV.
        output_path (str): Caminho de saída para o arquivo Parquet.
    """
    try:
        # Lê o arquivo CSV para um DataFrame
        df = pd.read_csv(csv_path)

        # Converte o DataFrame para uma Tabela PyArrow
        table = pa.Table.from_pandas(df)

        # Escreve o Parquet com compressão automática
        pq.write_table(table, output_path)

        # Obtém o tamanho do arquivo Parquet gerado
        tamanho_arquivo = os.path.getsize(output_path)

        # Exibe o tamanho do arquivo
        print(f"Arquivo Parquet com compressão automática criado. Tamanho do arquivo: {convert_bytes(tamanho_arquivo)}")

    except Exception as e:
        print(f"Erro ao criar o Parquet: {e}")

# Exemplo de uso
criar_parquet_com_compressao_automatica(csv_file_path, f'{output_dir}/PFW_2021_public_compressao_automatica.parquet')


Arquivo Parquet com compressão automática criado. Tamanho do arquivo: 3.2 MB


Explicação: Quando você não especifica a compressão, o PyArrow tentará usar o melhor método disponível para compressão. Dependendo do ambiente e da instalação, isso pode ser Snappy ou outro tipo.

## Usando Compressão snappy e Compactação de Colunas Individualmente

Se você tem um grande número de colunas no seu DataFrame, pode usar diferentes algoritmos de compressão para cada coluna. Aqui está como isso pode ser feito:

In [29]:
def criar_parquet_com_compressao_individual(csv_path, output_path):
    """
    Lê um CSV, converte para Parquet, aplica compressão individualmente por coluna.

    Args:
        csv_path (str): Caminho para o arquivo CSV.
        output_path (str): Caminho de saída para o arquivo Parquet.
    """
    try:
        # Lê o arquivo CSV para um DataFrame
        df = pd.read_csv(csv_path)

        # Converte o DataFrame para uma Tabela PyArrow
        table = pa.Table.from_pandas(df)

        # Especifica compressão diferente por coluna (aqui, apenas como exemplo)
        compression_opts = {
            'Nome': 'snappy',  # Compressão snappy para a coluna 'Nome'
            'Idade': 'gzip',   # Compressão gzip para a coluna 'Idade'
            'Salario': 'zstd', # Compressão zstd para a coluna 'Salario'
            'Data_Admissao': 'brotli', # Compressão brotli para a coluna 'Data_Admissao'
        }

        # Escreve o Parquet com compressão por coluna
        pq.write_table(table, output_path, compression=compression_opts)

        # Obtém o tamanho do arquivo Parquet gerado
        tamanho_arquivo = os.path.getsize(output_path)

        # Exibe o tamanho do arquivo
        print(f"Arquivo Parquet com compressão individual criada. Tamanho do arquivo: {convert_bytes(tamanho_arquivo)}")

    except Exception as e:
        print(f"Erro ao criar o Parquet: {e}")

# Exemplo de uso
criar_parquet_com_compressao_individual(csv_file_path, f'{output_dir}/compressao_colunas.parquet')


Arquivo Parquet com compressão individual criada. Tamanho do arquivo: 4.5 MB


Explicação: Nesse exemplo, usamos diferentes tipos de compressão em cada coluna. A coluna Nome usará snappy, Idade usará gzip, e assim por diante. Essa técnica pode ser útil quando você sabe que alguns dados (como strings) podem ser compactados mais eficientemente com certos algoritmos.

## Script de Comparação de Compressões Parquet

In [35]:
# Função para salvar arquivo Parquet com diferentes tipos de compressão
def salvar_parquet_com_compressao(csv_path, output_path, compression):
    """
    Função que converte um CSV para Parquet com a compressão especificada.

    Args:
        csv_path (str): Caminho para o arquivo CSV de entrada.
        output_path (str): Caminho de saída para o arquivo Parquet.
        compression (str): Tipo de compressão (snappy, gzip, brotli, zstd).
    """
    df = pd.read_csv(csv_path)
    table = pa.Table.from_pandas(df)

    # Medir tempo de execução para compressão
    start_time = time.time()
    pq.write_table(table, output_path, compression=compression)
    end_time = time.time()

    # Exibir o tempo de execução
    tempo_execucao = end_time - start_time
    tamanho_arquivo = os.path.getsize(output_path)

    print(f"Compressão {compression}:\n"
          f"Tempo de execução: {tempo_execucao:.4f} segundos\n"
          f"Tamanho do arquivo: {convert_bytes(tamanho_arquivo)}\n")

# Tipos de compressão e suas características:
tipos_compressao = {
    "snappy": "Snappy é rápido e proporciona boa performance de leitura e escrita. "
              "Ideal para grandes volumes de dados quando a velocidade de leitura/escrita é mais importante "
              "do que a taxa de compressão. Usado comumente no Hadoop e Spark.",

    "gzip": "Gzip oferece uma compressão mais forte que o Snappy, mas com um custo maior em termos de velocidade "
            "de leitura e escrita. Ideal quando é necessário reduzir o tamanho do arquivo e quando o tempo "
            "de leitura/gravação não é crítico.",

    "brotli": "Brotli é uma compressão moderna que oferece taxas de compressão superiores ao Gzip, "
              "mas tende a ser mais lenta. É uma boa escolha quando se deseja um bom equilíbrio entre "
              "compressão eficiente e desempenho.",

    "zstd": "Zstd (Zstandard) oferece alta compressão e performance comparável ao Snappy. "
            "Ideal para quando se deseja a melhor taxa de compressão com tempos razoáveis de leitura/escrita, "
            "muito usado em ambientes modernos."
}

# Exibindo as informações sobre cada tipo de compressão
print("Tipos de Compressão e quando usar:\n")
for tipo, descricao in tipos_compressao.items():
    print(f"{tipo.upper()}: {descricao}\n")

# Caminhos dos arquivos Parquet com diferentes compressões
output_parquet_snappy = os.path.join(output_dir, 'snappy.parquet')
output_parquet_gzip = os.path.join(output_dir, 'gzip.parquet')
output_parquet_brotli = os.path.join(output_dir, 'brotli.parquet')
output_parquet_zstd = os.path.join(output_dir, 'zstd.parquet')

# Salvar arquivos Parquet com compressão de cada tipo
salvar_parquet_com_compressao(csv_file_path, output_parquet_snappy, 'snappy')
salvar_parquet_com_compressao(csv_file_path, output_parquet_gzip, 'gzip')
salvar_parquet_com_compressao(csv_file_path, output_parquet_brotli, 'brotli')
salvar_parquet_com_compressao(csv_file_path, output_parquet_zstd, 'zstd')


Tipos de Compressão e quando usar:

SNAPPY: Snappy é rápido e proporciona boa performance de leitura e escrita. Ideal para grandes volumes de dados quando a velocidade de leitura/escrita é mais importante do que a taxa de compressão. Usado comumente no Hadoop e Spark.

GZIP: Gzip oferece uma compressão mais forte que o Snappy, mas com um custo maior em termos de velocidade de leitura e escrita. Ideal quando é necessário reduzir o tamanho do arquivo e quando o tempo de leitura/gravação não é crítico.

BROTLI: Brotli é uma compressão moderna que oferece taxas de compressão superiores ao Gzip, mas tende a ser mais lenta. É uma boa escolha quando se deseja um bom equilíbrio entre compressão eficiente e desempenho.

ZSTD: Zstd (Zstandard) oferece alta compressão e performance comparável ao Snappy. Ideal para quando se deseja a melhor taxa de compressão com tempos razoáveis de leitura/escrita, muito usado em ambientes modernos.

Compressão snappy:
Tempo de execução: 0.0720 segundos
Tamanho d